# Panel Data Models
---

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from linearmodels import PooledOLS
from linearmodels import RandomEffects
from linearmodels import PanelOLS
from linearmodels import FirstDifferenceOLS

from stargazer.stargazer import Stargazer

In [2]:
# Import crime data from North Carolina between 1981 to 1987
data = pd.read_csv("data/crime.csv", index_col=0)
data

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lwfir,lwser,lwmfg,lwfed,lwsta,lwloc,lpctymle,lpctmin,ltaxpc,lmix
1,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.69763,...,5.607452,5.374044,5.434246,6.014619,5.464848,5.444450,-2.433870,3.006608,3.246399,-2.303407
2,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.87425,...,5.706707,5.444911,5.482013,6.039540,5.536862,5.467174,-2.449038,3.006608,3.213833,-2.272549
3,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.45144,...,5.736475,5.481292,5.597310,6.084157,5.522900,5.515765,-2.464036,3.006608,3.275311,-2.517281
4,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.84235,...,5.858180,5.531204,5.640985,6.129421,5.568077,5.577387,-2.478925,3.006608,3.289981,-2.544612
5,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.14034,...,5.948220,5.564850,5.700042,6.195282,5.639919,5.664972,-2.497306,3.006608,3.337204,-2.372487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,197,83,0.015575,0.226667,0.480392,0.428571,7.77,0.001073,0.869048,18.90585,...,5.540736,5.280478,5.545919,5.865476,5.844326,5.463408,-2.538060,1.697597,2.939471,-2.172773
627,197,84,0.013662,0.204188,1.410260,0.372727,10.11,0.001109,0.872024,22.70475,...,5.569252,5.261022,5.593186,5.846150,5.869890,5.508943,-2.548068,1.697597,3.122574,-2.145931
628,197,85,0.013086,0.180556,0.830769,0.333333,5.96,0.001054,0.875000,24.12361,...,5.604843,5.333961,5.631821,5.880086,5.871498,5.616807,-2.561072,1.697597,3.183191,-2.512306
629,197,86,0.012874,0.112676,2.250000,0.244444,7.68,0.001088,0.880952,24.98198,...,5.743947,5.371892,5.723879,5.931024,5.873919,5.685245,-2.580968,1.697597,3.218155,-2.580217


In [3]:
# Best practice: check dtypes
# There are two columns with object dtypes that should be further investigated
# BUT since we will not be using those columns in this notebook, 
# we can safely ignore them 

data.dtypes

county        int64
year          int64
crmrte      float64
prbarr      float64
prbconv     float64
prbpris     float64
avgsen      float64
polpc       float64
density     float64
taxpc       float64
region       object
smsa         object
pctmin      float64
wcon        float64
wtuc        float64
wtrd        float64
wfir        float64
wser        float64
wmfg        float64
wfed        float64
wsta        float64
wloc        float64
mix         float64
pctymle     float64
lcrmrte     float64
lprbarr     float64
lprbconv    float64
lprbpris    float64
lavgsen     float64
lpolpc      float64
ldensity    float64
lwcon       float64
lwtuc       float64
lwtrd       float64
lwfir       float64
lwser       float64
lwmfg       float64
lwfed       float64
lwsta       float64
lwloc       float64
lpctymle    float64
lpctmin     float64
ltaxpc      float64
lmix        float64
dtype: object

In [4]:
data.columns

Index(['county', 'year', 'crmrte', 'prbarr', 'prbconv', 'prbpris', 'avgsen',
       'polpc', 'density', 'taxpc', 'region', 'smsa', 'pctmin', 'wcon', 'wtuc',
       'wtrd', 'wfir', 'wser', 'wmfg', 'wfed', 'wsta', 'wloc', 'mix',
       'pctymle', 'lcrmrte', 'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen',
       'lpolpc', 'ldensity', 'lwcon', 'lwtuc', 'lwtrd', 'lwfir', 'lwser',
       'lwmfg', 'lwfed', 'lwsta', 'lwloc', 'lpctymle', 'lpctmin', 'ltaxpc',
       'lmix'],
      dtype='object')

# 1. Random Effects, Fixed Effects, and First Differences
---

## Comments:
- In notebook 1_introduction, I wrote the code for both non-formula and formula APIs to running regressions in statsmodels.
- However, for simplicity and comparability with R code, I will only use the formula API from this notebook onwards.
- The main library from this notebook onwards is linearmodels.
- Linearmodels is a library that extends statsmodels with econometric models.

In [5]:
# Important! You must set a multi-index (entity, time) for panel data methods in linearmodels to work

data = data.set_index(['county','year'])
data

crmrte    prbarr   prbconv   prbpris  avgsen     polpc  \
county year                                                             
1      81    0.039885  0.289696  0.402062  0.472222    5.61  0.001787   
       82    0.038345  0.338111  0.433005  0.506993    5.59  0.001767   
       83    0.030305  0.330449  0.525703  0.479705    5.80  0.001836   
       84    0.034726  0.362525  0.604706  0.520104    6.89  0.001886   
       85    0.036573  0.325395  0.578723  0.497059    6.55  0.001924   
...               ...       ...       ...       ...     ...       ...   
197    83    0.015575  0.226667  0.480392  0.428571    7.77  0.001073   
       84    0.013662  0.204188  1.410260  0.372727   10.11  0.001109   
       85    0.013086  0.180556  0.830769  0.333333    5.96  0.001054   
       86    0.012874  0.112676  2.250000  0.244444    7.68  0.001088   
       87    0.014193  0.207595  1.182930  0.360825   12.23  0.001186   

              density     taxpc   region smsa  ...     lwfir     lwser  \
county year                                    ...                       
1      81    2.307159  25.69763  central   no  ...  5.607452  5.374044   
       82    2.330254  24.87425  central   no  ...  5.706707  5.444911   
       83    2.341801  26.45144  central   no  ...  5.736475  5.481292   
       84    2.346420  26.84235  central   no  ...  5.858180  5.531204   
       85    2.364896  28.14034  central   no  ...  5.948220  5.564850   
...               ...       ...      ...  ...  ...       ...       ...   
197    83    0.869048  18.90585     west   no  ...  5.540736  5.280478   
       84    0.872024  22.70475     west   no  ...  5.569252  5.261022   
       85    0.875000  24.12361     west   no  ...  5.604843  5.333961   
       86    0.880952  24.98198     west   no  ...  5.743947  5.371892   
       87    0.889881  25.95258     west   no  ...  5.852614  5.360449   

                lwmfg     lwfed     lwsta     lwloc  lpctymle   lpctmin  \
county year                                                               
1      81    5.434246  6.014619  5.464848  5.444450 -2.433870  3.006608   
       82    5.482013  6.039540  5.536862  5.467174 -2.449038  3.006608   
       83    5.597310  6.084157  5.522900  5.515765 -2.464036  3.006608   
       84    5.640985  6.129421  5.568077  5.577387 -2.478925  3.006608   
       85    5.700042  6.195282  5.639919  5.664972 -2.497306  3.006608   
...               ...       ...       ...       ...       ...       ...   
197    83    5.545919  5.865476  5.844326  5.463408 -2.538060  1.697597   
       84    5.593186  5.846150  5.869890  5.508943 -2.548068  1.697597   
       85    5.631821  5.880086  5.871498  5.616807 -2.561072  1.697597   
       86    5.723879  5.931024  5.873919  5.685245 -2.580968  1.697597   
       87    5.777405  5.970547  5.954930  5.726359 -2.601006  1.697597   

               ltaxpc      lmix  
county year                      
1      81    3.246399 -2.303407  
       82    3.213833 -2.272549  
       83    3.275311 -2.517281  
       84    3.289981 -2.544612  
       85    3.337204 -2.372487  
...               ...       ...  
197    83    2.939471 -2.172773  
       84    3.122574 -2.145931  
       85    3.183191 -2.512306  
       86    3.218155 -2.580217  
       87    3.256271 -2.694627  

[630 rows x 42 columns]

## 1.1. Pooled OLS

In [6]:
# Naive OLS assuming i.i.d. data

mod_OLS = PooledOLS.from_formula("crmrte ~ 1 + density + taxpc + wcon + pctmin", data=data)
res_OLS = mod_OLS.fit()
res_OLS.summary

Dep. Variable:,crmrte,R-squared:,0.5369
Estimator:,PooledOLS,R-squared (Between):,0.6178
No. Observations:,630,R-squared (Within):,-0.0076
Date:,"Thu, Mar 11 2021",R-squared (Overall):,0.5369
Time:,10:57:52,Log-likelihood,1875.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,181.12
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(4,625)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,181.12


## 1.2. Random Effects

In [7]:
mod_randeff = RandomEffects.from_formula("crmrte ~ 1 + density + taxpc + wcon + pctmin", data=data)
res_randeff = mod_randeff.fit()
res_randeff.summary

Dep. Variable:,crmrte,R-squared:,0.1804
Estimator:,RandomEffects,R-squared (Between):,0.6137
No. Observations:,630,R-squared (Within):,0.0015
Date:,"Thu, Mar 11 2021",R-squared (Overall):,0.5345
Time:,10:57:52,Log-likelihood,2231.3
Cov. Estimator:,Unadjusted,,
,,F-statistic:,34.396
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(4,625)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,34.396


## 1.3. Fixed Effects

In [8]:
# Entity fixed effects

entity_fe_formula = "crmrte ~ density + taxpc + wcon + pctmin + EntityEffects"
mod_entity_fe = PanelOLS.from_formula(entity_fe_formula, data=data, drop_absorbed=True)
res_entity_fe = mod_entity_fe.fit()
res_entity_fe.summary

/home/topher-lo/repos/PyEC333/ec333_env/lib/python3.8/site-packages/linearmodels/panel/model.py:1743: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

pctmin

  warnings.warn(


Dep. Variable:,crmrte,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-0.0345
No. Observations:,630,R-squared (Within):,0.0086
Date:,"Thu, Mar 11 2021",R-squared (Overall):,-0.0331
Time:,10:57:52,Log-likelihood,2280.2
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.5605
Entities:,90,P-value,0.1980
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,1.5605


In [9]:
# Entity and time period fixed effects

twoways_fe_formula = "crmrte ~ density + taxpc + wcon + pctmin + TimeEffects + EntityEffects"
mod_twoways_fe = PanelOLS.from_formula(twoways_fe_formula, data=data, drop_absorbed=True)
res_twoways_fe = mod_twoways_fe.fit()
res_twoways_fe.summary

Dep. Variable:,crmrte,R-squared:,0.0083
Estimator:,PanelOLS,R-squared (Between):,0.2240
No. Observations:,630,R-squared (Within):,0.0066
Date:,"Thu, Mar 11 2021",R-squared (Overall):,0.2170
Time:,10:57:52,Log-likelihood,2296.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.4778
Entities:,90,P-value,0.2196
Avg Obs:,7.0000,Distribution:,"F(3,531)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,1.4778


## 1.4. First Differences

In [10]:
mod_fd = FirstDifferenceOLS.from_formula("crmrte ~ density + taxpc + wcon", data=data)
res_fd = mod_fd.fit()
res_fd.summary

Dep. Variable:,crmrte,R-squared:,0.0019
Estimator:,FirstDifferenceOLS,R-squared (Between):,0.5954
No. Observations:,540,R-squared (Within):,0.0015
Date:,"Thu, Mar 11 2021",R-squared (Overall):,0.5764
Time:,10:57:52,Log-likelihood,1779.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.3358
Entities:,90,P-value,0.7995
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,0.3358


# 2. Standard Errors
---

### Note:
- Three covariance estiminators are supported in linearmodels:
    1. White's (1980) robust standard errors
    2. Clustered standard errors (entity, time, two-way)
    3. Driscoll-Kraay HAC standard errors
    
Reference: https://bashtage.github.io/linearmodels/doc/panel/models.html

In [11]:
# warning: this is probably bad
# White's (1980) robust standard errors

res_entity_fe_robust = mod_entity_fe.fit(cov_type="robust")
res_entity_fe_robust.summary

Dep. Variable:,crmrte,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-0.0345
No. Observations:,630,R-squared (Within):,0.0086
Date:,"Thu, Mar 11 2021",R-squared (Overall):,-0.0331
Time:,10:57:52,Log-likelihood,2280.2
Cov. Estimator:,Robust,,
,,F-statistic:,1.5605
Entities:,90,P-value,0.1980
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,4.9931


In [12]:
# Clustered by entity (fixed effects)

res_fe_clustered_entity = mod_entity_fe.fit(cov_type="clustered", cluster_entity=True)
res_fe_clustered_entity.summary

Dep. Variable:,crmrte,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-0.0345
No. Observations:,630,R-squared (Within):,0.0086
Date:,"Thu, Mar 11 2021",R-squared (Overall):,-0.0331
Time:,10:57:52,Log-likelihood,2280.2
Cov. Estimator:,Clustered,,
,,F-statistic:,1.5605
Entities:,90,P-value,0.1980
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,5.0931


In [13]:
# Clustered by entity (first difference)

res_fd_clustered_entity = mod_fd.fit(cov_type="clustered", cluster_entity=True)
res_fd_clustered_entity.summary

Dep. Variable:,crmrte,R-squared:,0.0019
Estimator:,FirstDifferenceOLS,R-squared (Between):,0.5954
No. Observations:,540,R-squared (Within):,0.0015
Date:,"Thu, Mar 11 2021",R-squared (Overall):,0.5764
Time:,10:57:53,Log-likelihood,1779.7
Cov. Estimator:,Clustered,,
,,F-statistic:,0.3358
Entities:,90,P-value,0.7995
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,4.8662


In [14]:
# Clustered by time (fixed effects)

res_fe_clustered_time = mod_entity_fe.fit(cov_type="clustered", cluster_time=True)
res_fe_clustered_time.summary

Dep. Variable:,crmrte,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-0.0345
No. Observations:,630,R-squared (Within):,0.0086
Date:,"Thu, Mar 11 2021",R-squared (Overall):,-0.0331
Time:,10:57:53,Log-likelihood,2280.2
Cov. Estimator:,Clustered,,
,,F-statistic:,1.5605
Entities:,90,P-value,0.1980
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,8.9901


In [15]:
# Clustered by time (first difference)

res_fd_clustered_time = mod_fd.fit(cov_type="clustered", cluster_time=True)
res_fd_clustered_time.summary

Dep. Variable:,crmrte,R-squared:,0.0019
Estimator:,FirstDifferenceOLS,R-squared (Between):,0.5954
No. Observations:,540,R-squared (Within):,0.0015
Date:,"Thu, Mar 11 2021",R-squared (Overall):,0.5764
Time:,10:57:53,Log-likelihood,1779.7
Cov. Estimator:,Clustered,,
,,F-statistic:,0.3358
Entities:,90,P-value,0.7995
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,4.5024


In [16]:
# Newey-West
# WARNING: linearmodels currently does not implement Newey-West SEs
# But statsmodels does! 
# The following code implements a entity-dummies fixed effects regression
# Note: you must include "-1" into the formula to remove the intercept

data_noindex = data.reset_index()
mod_fe_NW = sm.formula.ols("crmrte ~ density + taxpc + wcon + C(county) -1", data=data_noindex)
res_fe_NW = mod_fe_NW.fit(cov_type="HAC", cov_kwds={"maxlags":1})
res_fe_NW.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 crmrte   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 11 Mar 2021   Prob (F-statistic):                nan
Time:                        10:57:53   Log-Likelihood:                 2280.2
No. Observations:                 630   AIC:                            -4374.
Df Residuals:                     537   BIC:                            -3961.
Df Model:                          92                                         
Covariance Type:                  HAC                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
C(county)[1]       0.0373      0.011      3.441      0.001       0.016       0.059
C(county)[3]       0.0154      0.004      3.495      0.000       0.007       0.024
C(county)[5]       0.0112      0.002      6.328      0.000       0.008       0.015
C(county)[7]       0.0206      0.003      8.062      0.000       0.016       0.026
C(county)[9]       0.0108      0.002      4.462      0.000       0.006       0.016
C(county)[11]      0.0160      0.003      6.014      0.000       0.011       0.021
C(county)[13]      0.0313      0.002     14.239      0.000       0.027       0.036
C(county)[15]      0.0155      0.002      8.457      0.000       0.012       0.019
C(county)[17]      0.0193      0.002      9.367      0.000       0.015       0.023
C(county)[19]      0.0126      0.004      3.442      0.001       0.005       0.020
C(county)[21]      0.0406      0.011      3.538      0.000       0.018       0.063
C(county)[23]      0.0292      0.007      4.348      0.000       0.016       0.042
C(county)[25]      0.0343      0.012      2.956      0.003       0.012       0.057
C(county)[27]      0.0351      0.007      5.298      0.000       0.022       0.048
C(county)[33]      0.0148      0.002      6.830      0.000       0.011       0.019
C(county)[35]      0.0447      0.013      3.478      0.001       0.020       0.070
C(county)[37]      0.0203      0.002      9.452      0.000       0.016       0.025
C(county)[39]      0.0136      0.002      6.172      0.000       0.009       0.018
C(county)[41]      0.0226      0.005      5.003      0.000       0.014       0.031
C(county)[45]      0.0360      0.008      4.377      0.000       0.020       0.052
C(county)[47]      0.0296      0.002     13.010      0.000       0.025       0.034
C(county)[49]      0.0340      0.005      7.111      0.000       0.025       0.043
C(county)[51]      0.0716      0.018      3.935      0.000       0.036       0.107
C(county)[53]      0.0153      0.003      5.737      0.000       0.010       0.021
C(county)[55]      0.0525      0.006      8.788      0.000       0.041       0.064
C(county)[57]      0.0312      0.010      3.079      0.002       0.011       0.051
C(county)[59]      0.0172      0.004      4.019      0.000       0.009       0.026
C(county)[61]      0.0195      0.002      8.000      0.000       0.015       0.024
C(county)[63]      0.0794      0.025      3.130      0.002       0.030       0.129
C(county)[65]      0.0559      0.006      9.405      0.000       0.044       0.068
C(county)[67]      0.0638      0.029      2.163      0.031       0.006       0.122
C(county)[69]      0.0138      0.003      5.043      0.000       0.008       0.019
C(county)[71]      0.0639      0.022      2.843      0.004       0.020       0.108
C(county)[77]      0.0375      0.003     11.580      0.000       0.031       0.044
C(county)[79

In [17]:
# Driscoll-Kraay HAC

res_fe_HAC = mod_entity_fe.fit(cov_type="kernel", kernel="andrews")
res_fe_HAC.summary

Dep. Variable:,crmrte,R-squared:,0.0086
Estimator:,PanelOLS,R-squared (Between):,-0.0345
No. Observations:,630,R-squared (Within):,0.0086
Date:,"Thu, Mar 11 2021",R-squared (Overall):,-0.0331
Time:,10:57:53,Log-likelihood,2280.2
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,1.5605
Entities:,90,P-value,0.1980
Avg Obs:,7.0000,Distribution:,"F(3,537)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,30.212
